Variational message passing demo
===

ForneyLab comes with support for variational message passing (VMP) (Dauwels, 2007). In this demo we illustrate VMP by estimating the mean and precision (inverse variance) of i.i.d. samples drawn from a Gaussian distribution. The model is easily defined as 
\begin{align*}
    y_i \sim \mathcal{N}(m, w^{-1})
\end{align*}
The factor graph below shows our generative model.

```
-------------> = ---> (w)
               | 
---> = --------|----> (m)
     |         |
     ---> N <---
          | 
        (y_i)
```

Variational inference approximates the posterior over $m$ and $w$ by a recognition distribution. We minimize the KL divergence between the exact posterior and the recognition distribution by variational message passing. 

In [1]:
using(ForneyLab)

n = 5

g = FactorGraph()

# Priors
m ~ GaussianMeanVariance(constant(0.0), constant(100.0))
w ~ Gamma(constant(0.01), constant(0.01))

# Observarion model
for i = 1:n
    y_i ~ GaussianMeanPrecision(m, w)
    placeholder(y_i, :y, index=i)
end

# Assign id for ease of lookup
m.id = :m
w.id = :w
;

With the model defined, we can now specify the recognition factorization, generate a schedule for each recognition factor.

In [2]:
# Specify recognition factorization
q_m = RecognitionFactor(m)
q_w = RecognitionFactor(w)

# Generate schedules
schedule_q_m = variationalSchedule(q_m)
schedule_q_w = variationalSchedule(q_w)
;

We can inspect a subgraph and the generated schedule

In [3]:
ForneyLab.draw(q_m, schedule=schedule_q_m)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.36.0 (20140111.2315)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 11401858864864087643 -->
 11401858864864087643 
 
 𝒩 
 gaussian_4 
 
<!-- 4975737540297414028 -->
 4975737540297414028 
 
 = 
 equ_variable_1_4 
 
<!-- 15134704083454708312 -->
 15134704083454708312 
 
 𝒩 
 gaussian_5 
 
<!-- 4975737540297414028--15134704083454708312 -->
 4975737540297414028--15134704083454708312 
 
 1 mean 
 ((2)) 
 1 1 
 
<!-- 7788808848379548158 -->
 7788808848379548158 
 
 = 
 equ_variable_1_3 
 
<!-- 4975737540297414028--7788808848379548158 -->
 4975737540297414028--7788808848379548158 
 
 3 3 
 2 2 
 (4) 
 
<!-- 18243915053181234126 -->
 18243915053181234126 
 
 𝒩 
 gaussian_6 
 
<!-- 4975737540297414028--18243915053181234126 -->
 4975737540297414028--18243915053181234126 
 
 1 mean 
 ((3)) 
 3 3 
 
<!-- 16461731861220907175 -->
 16461731861220907175 
 
 = 
 equ_variable_1_2 
 
<!-- 3606736376428855940 -->
 3606736376428855940 
 
 = 
 equ_variable_1_1 
 
<!-- 16461731861220907175--3606736376428855940 -->
 16461731861220907175--3606736376428855940 
 
 3 3 
 2 2 
 (8) 
 
<!-- 12924836573515514319 -->
 12924836573515514319 
 
 𝒩 
 gaussian_3 
 
<!-- 16461731861220907175--12924836573515514319 -->
 16461731861220907175--12924836573515514319 
 
 1 mean 
 ((7)) 
 1 1 
 
<!-- 3559487250188741623 -->
 3559487250188741623 
 
 𝒩 
 gaussian_1 
 
<!-- 3746688169167547672 -->
 3746688169167547672 
 
<!-- 3559487250188741623--3746688169167547672 -->
 3559487250188741623--3746688169167547672 
 
 1 out 
 2 variance 
 
<!-- 14505614521621519613 -->
 14505614521621519613 
 
<!-- 3559487250188741623--14505614521621519613 -->
 3559487250188741623--14505614521621519613 
 
 1 out 
 1 mean 
 
<!-- 14402671564515670611 -->
 14402671564515670611 
 
 𝒩 
 gaussian_2 
 
<!-- 7788808848379548158--11401858864864087643 -->
 7788808848379548158--11401858864864087643 
 
 1 mean 
 ((5)) 
 1 1 
 
<!-- 7788808848379548158--16461731861220907175 -->
 7788808848379548158--16461731861220907175 
 
 3 3 
 2 2 
 (6) 
 
<!-- 3606736376428855940--3559487250188741623 -->
 3606736376428855940--3559487250188741623 
 
 3 out 
 ((1)) 
 1 1 
 (10) 
 
<!-- 3606736376428855940--14402671564515670611 -->
 3606736376428855940--14402671564515670611 
 
 1 mean 
 ((9)) 
 2 2 
 
<!-- 9831009233793930946 -->
 9831009233793930946 
 
<!-- 9831009233793930946--18243915053181234126 -->
 9831009233793930946--18243915053181234126 
 
 3 out 
 1 out 
 
<!-- 6983668894954939547 -->
 6983668894954939547 
 
<!-- 6983668894954939547--14402671564515670611 -->
 6983668894954939547--14402671564515670611 
 
 2 precision 
 2 2 
 
<!-- 157566049068009330 -->
 157566049068009330 
 
<!-- 157566049068009330--12924836573515514319 -->
 157566049068009330--12924836573515514319 
 
 3 out 
 1 out 
 
<!-- 10288490278290575838 -->
 10288490278290575838 
 
<!-- 10288490278290575838--15134704083454708312 -->
 10288490278290575838--15134704083454708312 
 
 3 out 
 1 out 
 
<!-- 14898714238423743861 -->
 14898714238423743861 
 
<!-- 14898714238423743861--14402671564515670611 -->
 14898714238423743861--14402671564515670611 
 
 3 out 
 1 out 
 
<!-- 9705399543679859492 -->
 9705399543679859492 
 
<!-- 9705399543679859492--12924836573515514319 -->
 9705399543679859492--12924836573515514319 
 
 2 precision 
 1 1 
 
<!-- 5028285900334611621 -->
 5028285900334611621 
 
<!-- 5028285900334611621--11401858864864087643 -->
 5028285900334611621--11401858864864087643 
 
 3 out 
 1 out 
 
<!-- 7849784095382315750 -->
 7849784095382315750 
 
<!-- 7849784095382315750--15134704083454708312 -->
 7849784095382315750--15134704083454708312 
 
 2 precision 
 1 1 
 
<!-- 7849784095382315750--18243915053181234126 -->
 7849784095382315750--18243915053181234126 
 
 2 precision 
 3 3 
 
<!-- 14926557405414277074 -->
 14926557405414277074 
 
<!-- 14926557405414277074-

In [4]:
# Convert schedules to Julia executable code
algo_q_m = messagePassingAlgorithm(schedule_q_m, m, name="M")
algo_q_w = messagePassingAlgorithm(schedule_q_w, w, name="W")

# Inspect the algorithm code
println(algo_q_m)

function stepM!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(10))

messages[1] = ruleVBGaussianMeanVariance3(ProbabilityDistribution(PointMass, m=0.0), ProbabilityDistribution(PointMass, m=100.0), nothing)
messages[2] = ruleVBGaussianMeanPrecision1(nothing, marginals[:w], ProbabilityDistribution(PointMass, m=data[:y][4]))
messages[3] = ruleVBGaussianMeanPrecision1(nothing, marginals[:w], ProbabilityDistribution(PointMass, m=data[:y][5]))
messages[4] = ruleSPEqualityGaussian(messages[2], nothing, messages[3])
messages[5] = ruleVBGaussianMeanPrecision1(nothing, marginals[:w], ProbabilityDistribution(PointMass, m=data[:y][3]))
messages[6] = ruleSPEqualityGaussian(messages[5], nothing, messages[4])
messages[7] = ruleVBGaussianMeanPrecision1(nothing, marginals[:w], ProbabilityDistribution(PointMass, m=data[:y][2]))
messages[8] = ruleSPEqualityGaussian(messages[7], nothing, messages[6])
messages[9] = ruleVBGaussianMeanPrecision1(nothing, marginals[:w], Probabi

In [5]:
# Inspect the algorithm code
println(algo_q_w)

function stepW!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(10))

messages[1] = ruleVBGamma3(ProbabilityDistribution(PointMass, m=0.01), ProbabilityDistribution(PointMass, m=0.01), nothing)
messages[2] = ruleVBGaussianMeanPrecision2(marginals[:m], nothing, ProbabilityDistribution(PointMass, m=data[:y][4]))
messages[3] = ruleVBGaussianMeanPrecision2(marginals[:m], nothing, ProbabilityDistribution(PointMass, m=data[:y][5]))
messages[4] = ruleSPEqualityGamma(messages[2], nothing, messages[3])
messages[5] = ruleVBGaussianMeanPrecision2(marginals[:m], nothing, ProbabilityDistribution(PointMass, m=data[:y][3]))
messages[6] = ruleSPEqualityGamma(messages[5], nothing, messages[4])
messages[7] = ruleVBGaussianMeanPrecision2(marginals[:m], nothing, ProbabilityDistribution(PointMass, m=data[:y][2]))
messages[8] = ruleSPEqualityGamma(messages[7], nothing, messages[6])
messages[9] = ruleVBGaussianMeanPrecision2(marginals[:m], nothing, ProbabilityDistribution(PointMa

We can now iteratively execute the updates for each factor, and inspect the results.

In [6]:
eval(parse(algo_q_m))
eval(parse(algo_q_w))

# Toy dataset
m_true = 3.0
w_true = 4.0
y = sqrt(1/w_true)*randn(n) + m_true
data = Dict(:y => y)

# Initial recognition distributions
marginals = Dict(:m => vague(ProbabilityDistribution{Gaussian}),
                 :w => vague(ProbabilityDistribution{Gamma}))

n_its = 2*n
for i = 1:n_its
   stepM!(data, marginals)
   stepW!(data, marginals)    
end
;

In [7]:
# Inspect the results
println("True mean: $(m_true)")
println("True precision: $(w_true)")
println("Number of samples: $(n)")
println("Sample mean: $(round(mean(y),2))")
println("Sample precision: $(round(1/var(y),2))")
println("\n----- Estimation after $(n_its) VMP updates -----")
println("Mean estimate: $(round(mean(marginals[:m]),2)), with variance $(round(var(marginals[:m]),2))")
println("Precision estimate: $(round(mean(marginals[:w]),2)), with variance $(round(var(marginals[:w]),2))")

True mean: 3.0
True precision: 4.0
Number of samples: 5
Sample mean: 2.98
Sample precision: 5.02

----- Estimation after 10 VMP updates -----
Mean estimate: 2.98, with variance 0.04
Precision estimate: 4.92, with variance 9.64
